In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict
from collections import Counter
from pprint import pprint
import pickle
import datetime
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import svm
from itertools import cycle
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as esw
# import enchant
from collections import Counter
from sklearn.metrics import pairwise_distances
from scipy.sparse import coo_matrix

import pickle

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from scipy.stats import zscore
import xgboost
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.metrics import explained_variance_score



In [4]:
def open_range_of_dicts(dict_name, last_number, an_orient='columns', bool_ignore_index=True):
    df_list = []
    for i in range(0, last_number+1):
        try:
            file = open("{}{}.pkl".format(dict_name, i),'rb')
            data_dict = pickle.load(file)
            file.close()
        except: 
            continue # In case there was an error and a specific pickle number does not exist
        df_list.append(pd.DataFrame.from_dict(data_dict, orient=an_orient))
    df = pd.concat(df_list, ignore_index=bool_ignore_index)
    return df

In [5]:
house_data = open_range_of_dicts('house_data_dict', 90, 'index', False)
house_data = house_data[~house_data.index.duplicated(keep='last')]
house_data = house_data.infer_objects()
house_data = house_data.applymap(lambda x: np.nan if x == 'nan' else x)
house_data = house_data.applymap(lambda x: np.nan if x == '—' else x)
house_data = house_data.applymap(lambda x: True if x == 'Yes' else x)
house_data = house_data.applymap(lambda x: False if x == 'No' else x)

keep = house_data[['latitude', 'longitude', 'description']]

for col in house_data.columns:
    cols = col.split(",")

    for new_col in cols:
        if new_col.replace(":", "").strip().lower() not in house_data.columns:
            house_data[new_col.replace(":", "").strip().lower()] = house_data[col]
        if new_col.replace(":", "").strip().lower() in house_data.columns:
            house_data[new_col.replace(":", "").strip().lower()] = house_data[col].combine(house_data[new_col.replace(":", "").strip().lower()], lambda x1, x2: x2 if x1 is np.nan else x1)
    house_data = house_data.drop(col, axis=1)


for col in house_data.columns:
    house_data[col] = pd.to_numeric(house_data[col], errors='ignore')

house_data = house_data.join(keep, how='outer')

/Users/jackselbo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [6]:
house_data.columns

Index(['apn', 'accessible', 'area current # sold homes',
       'area current avg. # offers', 'area current avg. down payment',
       'area current median $ / sq. ft.', 'area current median list price',
       'area current median sale / list', 'basement', 'baths', 'beds',
       'bike score®', 'building', 'built', 'community', 'county', 'features',
       'finished sq. ft.', 'garage', 'hoa dues', 'listing#', 'lot size',
       'mls#', 'parking spaces', 'property type', 'sq. ft.', 'stories',
       'style', 'total sq. ft.', 'transit score®', 'type',
       'unfinished sq. ft.', 'walk score®', 'year built', 'year renovated',
       'addresslocality', 'addressregion', 'postalcode', 'streetaddress',
       'latitude', 'longitude', 'description'],
      dtype='object')

In [7]:
house_data.shape

(7094, 42)

In [8]:
# Major columns, identified by data filled
# test_data.isnull().sum().sort_values(ascending=True)[0:50]/len(test_data)

identifiers = [
    'streetaddress',
    'addresslocality',
    'addressregion',
    'postalcode',
    'community',
    'latitude',
    'longitude'
]

house_details_quant = [
    'beds',
    'baths',
    'stories',
    'lot size',
    'year built',
    'year renovated',
    'transit score®',
    'bike score®',
    'walk score®',
    'hoa dues',
    'finished sq. ft.',
    'total sq. ft.', 
]

house_details_qual = [
    'style'
]

keep = house_data['description']

def clean_quant(old_df):
    df = old_df.copy()
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col].str.replace(" Sq. Ft.", "")
                                           .str.replace(",","")
                                           .str.replace(" / 100", "")
                                           .str.replace("$", "")
                                           .str.replace('/month', ""),
                                    errors='coerce'
                                    )
        except:
            None
    
    def float_to_datetime(x):
        try:
            y = datetime.datetime(year=int(x), month=1, days=1)
            return y
        except:
            return np.nan
    
    df['year renovated'] = pd.to_datetime(df['year renovated'].astype(str).str[0:4])
    df['year built'] = pd.to_datetime(df['year built'].astype(str).str[0:4])
    
    return df

def clean_qual(old_df):
    df = old_df.copy()
    cols = df.columns
    for col in cols:
        try:
            new_df = df[col].str.get_dummies(sep=', ').rename(lambda x: str(col) + "_" + x, axis='columns')
            df = df.drop(col, axis=1)
            df = pd.merge(df, new_df, left_index=True, right_index=True)
        except:
            pass
    
    df = df.fillna(0)
    
    return df

house_quant_df = clean_quant(house_data[house_details_quant])
house_qual_df = clean_qual(house_data[house_details_qual])
cleaned_house_data = pd.merge(pd.merge(house_data[identifiers],house_quant_df,left_index=True, right_index=True),house_qual_df,left_index=True, right_index=True)
cleaned_house_data = cleaned_house_data.join(keep, how='outer')

In [9]:
cleaned_house_data.head()

,streetaddress,addresslocality,addressregion,postalcode,community,latitude,longitude,beds,baths,stories,...,hoa dues,finished sq. ft.,total sq. ft.,style_Condo/Co-op,style_MasterPlanned,style_Multi-Family (2-4 Unit),style_Other,style_Single Family Residential,style_Vacant Land,description
https://www.redfin.com/IN/Indianapolis/1-Virginia-Ave-46204/unit-704/home/148374142,1 Virginia Ave #704,"Indianapolis,",IN,46204,Center SE,39.766032,-86.155980,NaN,NaN,NaN,...,647.0,NaN,NaN,1,0,0,0,0,0,Build out your dream condo - standards in list...
https://www.redfin.com/IN/Indianapolis/141-S-Meridian-St-46225/unit-404/home/82299357,141 S Meridian St #404,"Indianapolis,",IN,46225,Center SE,39.764481,-86.157962,2.0,1.5,1.0,...,862.0,2966.0,2966.0,1,0,0,0,0,0,If you are looking for a 3 bedroom spacious co...
https://www.redfin.com/IN/Indianapolis/141-S-Meridian-St-46225/unit-601/home/82299319,141 S Meridian St S #601,"Indianapolis,",IN,46225,Center SE,39.764481,-86.157962,2.0,1.5,1.0,...,550.0,2107.0,2107.0,1,0,0,0,0,0,"Location, views, style! This turn key condo ha..."
https://www.redfin.com/IN/Indianapolis/207-S-Meridian-St-46225/unit-4C/home/82298027,207 S Meridian St Unit 4C,"Indianapolis,",IN,46225,Center SE,39.764013,-86.157925,NaN,NaN,NaN,...,449.0,NaN,NaN,1,0,0,0,0,0,Exclusive Opportunity to live in the heart of ...
https://www.redfin.com/IN/Indianapolis/222-E-North-St-46204/unit-107/home/82331858,222 N East St #107,"Indianapolis,",IN,46204,Center NE,39.770519,-86.149371,NaN,NaN,NaN,...,398.0,NaN,NaN,1,0,0,0,0,0,Gorgeous unit in sought-after Landmark at Lock...


In [10]:
(house_quant_df.notnull().sum() / len(house_quant_df)).sort_values()

finished sq. ft.    0.174937
total sq. ft.       0.174937
hoa dues            0.321116
baths               0.409078
lot size            0.563152
beds                0.713561
stories             0.840992
year renovated      0.857203
year built          0.875247
transit score®      0.954469
bike score®         0.957711
walk score®         0.957711
dtype: float64

In [11]:
house_data.head()

,apn,accessible,area current # sold homes,area current avg. # offers,area current avg. down payment,area current median $ / sq. ft.,area current median list price,area current median sale / list,basement,baths,...,walk score®,year built,year renovated,addresslocality,addressregion,postalcode,streetaddress,latitude,longitude,description
https://www.redfin.com/IN/Indianapolis/1-Virginia-Ave-46204/unit-704/home/148374142,491112102005003101,NaN,3.0,1.0,20.0%,$253,"$432,500",95%,NaN,NaN,...,96 / 100,NaN,NaN,"Indianapolis,",IN,46204,1 Virginia Ave #704,39.766032,-86.155980,Build out your dream condo - standards in list...
https://www.redfin.com/IN/Indianapolis/141-S-Meridian-St-46225/unit-404/home/82299357,491101237050007101,NaN,3.0,1.0,20.0%,$253,"$432,500",95%,NaN,1.5,...,84 / 100,2006.0,2006.0,"Indianapolis,",IN,46225,141 S Meridian St #404,39.764481,-86.157962,If you are looking for a 3 bedroom spacious co...
https://www.redfin.com/IN/Indianapolis/141-S-Meridian-St-46225/unit-601/home/82299319,491101237050015101,NaN,3.0,1.0,20.0%,$253,"$432,500",95%,NaN,1.5,...,84 / 100,2005.0,2005.0,"Indianapolis,",IN,46225,141 S Meridian St S #601,39.764481,-86.157962,"Location, views, style! This turn key condo ha..."
https://www.redfin.com/IN/Indianapolis/207-S-Meridian-St-46225/unit-4C/home/82298027,491112238016006101,NaN,11.0,NaN,NaN,$109,"$199,450",94%,NaN,NaN,...,82 / 100,NaN,NaN,"Indianapolis,",IN,46225,207 S Meridian St Unit 4C,39.764013,-86.157925,Exclusive Opportunity to live in the heart of ...
https://www.redfin.com/IN/Indianapolis/222-E-North-St-46204/unit-107/home/82331858,491101240259006101,NaN,3.0,1.0,20.0%,$253,"$432,500",95%,NaN,NaN,...,94 / 100,NaN,NaN,"Indianapolis,",IN,46204,222 N East St #107,39.770519,-86.149371,Gorgeous unit in sought-after Landmark at Lock...


In [12]:
(cleaned_house_data.notnull().sum() / len(cleaned_house_data)).sort_values()

total sq. ft.                      0.174937
finished sq. ft.                   0.174937
hoa dues                           0.321116
baths                              0.409078
lot size                           0.563152
beds                               0.713561
stories                            0.840992
year renovated                     0.857203
year built                         0.875247
transit score®                     0.954469
walk score®                        0.957711
bike score®                        0.957711
community                          0.967014
description                        0.969411
longitude                          1.000000
latitude                           1.000000
postalcode                         1.000000
addressregion                      1.000000
addresslocality                    1.000000
style_Condo/Co-op                  1.000000
style_MasterPlanned                1.000000
style_Multi-Family (2-4 Unit)      1.000000
style_Other                     

In [13]:
cleaned_house_data.columns

Index(['streetaddress', 'addresslocality', 'addressregion', 'postalcode',
       'community', 'latitude', 'longitude', 'beds', 'baths', 'stories',
       'lot size', 'year built', 'year renovated', 'transit score®',
       'bike score®', 'walk score®', 'hoa dues', 'finished sq. ft.',
       'total sq. ft.', 'style_Condo/Co-op', 'style_MasterPlanned',
       'style_Multi-Family (2-4 Unit)', 'style_Other',
       'style_Single Family Residential', 'style_Vacant Land', 'description'],
      dtype='object')

In [14]:
cleaned_house_data.shape

(7094, 26)

In [15]:
# NLP

text = cleaned_house_data.description.str.replace('[^\sa-zA-Z]|_', '')
text_stop_words = [] + list(esw)

vectorizer = TfidfVectorizer(stop_words=text_stop_words,min_df=.01,max_df=.3)
transformed = vectorizer.fit_transform(text.values.astype(str))
corpus = pd.DataFrame(transformed.toarray(), columns=vectorizer.get_feature_names())
corpus.shape

(7094, 662)

In [16]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [17]:
def try_diff_num_topics(start=2,end=5, trans=None, vect=None):
    for n in range(start,end):
        print('')
        print(str(n)+' Topics')
        nmf = NMF(n)
        doc_topic = nmf.fit_transform(trans)
        ind = []
        for x in range(n): ind.append('component_'+str(x))
        topic_word = pd.DataFrame(nmf.components_.round(3), index = ind,columns = vect.get_feature_names())
        display_topics(nmf, vect.get_feature_names(), 9)
        
        print('')
        topid=[]
        for doc in doc_topic:
            topid.append(np.argmax(doc))
        print("Topics: "+str(n)+" "+str(Counter(topid)))
        
        print('')
        print("*********************************************")

In [18]:
try_diff_num_topics(10,30, transformed, vectorizer) #22


10 Topics

Topic  0
closet, walkin, suite, open, shower, floor, double, tub, ceiling

Topic  1
nan, yrs, flooring, foyer, fountain, formal, foot, flows, flow

Topic  2
property, investment, opportunity, sold, rental, needs, make, duplex, potential

Topic  3
description, plan, floor, open, split, construction, feet, bathrooms, offers

Topic  4
paint, flooring, updated, roof, water, heater, newer, windows, furnace

Topic  5
downtown, location, just, restaurants, located, close, minutes, square, fountain

Topic  6
living, family, yard, fenced, bedrooms, ranch, space, nice, dining

Topic  7
basement, floors, hardwood, original, finished, porch, updated, bungalow, broad

Topic  8
lot, corner, vacant, build, area, acre, available, square, opportunity

Topic  9
level, main, area, bedrooms, upper, lower, laundry, upstairs, living

Topics: 10 Counter({4: 1191, 6: 1081, 5: 955, 7: 933, 0: 865, 2: 864, 9: 577, 8: 317, 1: 217, 3: 94})

*********************************************

11 Topics

Top


Topic  0
closet, walkin, suite, shower, double, tub, open, vaulted, ceiling

Topic  1
nan, yrs, flooring, foyer, fountain, formal, foot, flows, flow

Topic  2
property, investment, opportunity, needs, rental, make, need, potential, tlc

Topic  3
description, plan, floor, open, split, construction, ranch, feet, total

Topic  4
water, heater, roof, newer, furnace, years, old, windows, ac

Topic  5
downtown, close, shopping, located, access, minutes, location, easy, park

Topic  6
living, space, family, yard, fenced, bedrooms, dining, storage, spacious

Topic  7
basement, floors, hardwood, original, finished, porch, bungalow, detached, broad

Topic  8
lot, corner, vacant, build, acre, available, ave, mature, city

Topic  9
level, main, lower, upper, bedrooms, upstairs, laundry, area, bonus

Topic  10
sold, asis, condition, rehab, property, seller, repairs, potential, package

Topic  11
square, fountain, build, opportunity, feet, construction, just, downtown, location

Topic  12
flooring,


Topic  0
condo, pool, community, unit, private, maintenance, clubhouse, patio, tennis

Topic  1
nan, yrs, flooring, foyer, fountain, formal, foot, flows, flow

Topic  2
property, rental, investment, income, seller, looking, bathroom, investors, starter

Topic  3
description, plan, split, total, commute, walkin, desirable, feet, perry

Topic  4
water, heater, roof, newer, furnace, years, old, windows, ac

Topic  5
downtown, location, close, shopping, located, minutes, access, park, restaurants

Topic  6
space, living, family, dining, bedrooms, plenty, offers, formal, spacious

Topic  7
basement, finished, broad, ripple, storage, bungalow, detached, porch, car

Topic  8
lot, corner, vacant, build, acre, available, ave, wooded, mature

Topic  9
level, main, lower, upper, laundry, upstairs, bedrooms, bonus, rm

Topic  10
sold, asis, condition, rehab, seller, package, repairs, sale, investors

Topic  11
floor, plan, open, st, views, th, best, contemporary, building

Topic  12
flooring, upd


Topic  0
tile, granite, stainless, updated, steel, cabinets, counter, tops, custom

Topic  1
nan, yrs, flooring, foyer, fountain, formal, foot, flows, flow

Topic  2
property, rental, investment, make, income, needs, looking, investors, seller

Topic  3
description, plan, split, total, commute, desirable, perry, feet, ranch

Topic  4
water, heater, newer, roof, furnace, windows, ac, updates, hvac

Topic  5
downtown, location, close, located, shopping, minutes, access, park, restaurants

Topic  6
space, living, bedrooms, plenty, offers, storage, dining, additional, spacious

Topic  7
basement, finished, broad, ripple, storage, bungalow, unfinished, detached, car

Topic  8
lot, corner, vacant, build, acre, available, ave, city, wooded

Topic  9
level, main, lower, upper, laundry, upstairs, bedrooms, bonus, baths

Topic  10
sold, asis, condition, rehab, seller, needs, potential, repairs, investor

Topic  11
floor, plan, open, st, views, th, best, contemporary, building

Topic  12
paint, 

In [19]:
# ex = dt.to_csv('raw.csv',index=False)
corpus.head()

,ac,access,acre,acres,add,added,addition,additional,adjacent,adorable,...,woodwork,work,workshop,wow,yard,year,years,youll,youre,yrs
0,0.0,0.000000,0.0,0.0,0.162078,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.123002,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
n = 10
nmf = NMF(n)
doc_topic = nmf.fit_transform(transformed)

ind = []
for x in range(n): ind.append('component_'+str(x))
topic_word = pd.DataFrame(nmf.components_.round(3), index = ind,columns = vectorizer.get_feature_names())
display_topics(nmf, vectorizer.get_feature_names(), 9)


Topic  0
closet, walkin, suite, open, shower, floor, double, tub, ceiling

Topic  1
nan, yrs, flooring, foyer, fountain, formal, foot, flows, flow

Topic  2
property, investment, opportunity, sold, rental, needs, make, duplex, potential

Topic  3
description, plan, floor, open, split, construction, feet, bathrooms, offers

Topic  4
paint, flooring, updated, roof, water, heater, newer, windows, furnace

Topic  5
downtown, location, just, restaurants, located, close, minutes, square, fountain

Topic  6
living, family, yard, fenced, bedrooms, ranch, space, nice, dining

Topic  7
basement, floors, hardwood, original, finished, porch, updated, bungalow, broad

Topic  8
lot, corner, vacant, build, area, acre, available, square, opportunity

Topic  9
level, main, area, bedrooms, upper, lower, laundry, upstairs, living


In [21]:
# Open data
price_data = open_range_of_dicts('price_history_dict', 90)

In [22]:
# Clean data types
def clean_data_types(df):
    # Clean up price column by removing non-numbers, convert to numeric
    df.loc[:, 'price'] = df['price'].replace('[\$—,\*]', '', regex=True)
    df.loc[:, 'price'] = pd.to_numeric(df['price'], errors='ignore')
    df = df.dropna(subset=['price'])
    
    # Clean up datetime
    df.loc[:, 'date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df.date = df.date.dt.to_period('M').dt.to_timestamp()

    # Checks
    print(df.dtypes)
    df.head()
    
    return df

price_data = clean_data_types(price_data)

date      datetime64[ns]
event             object
source            object
price            float64
url               object
dtype: object


/Users/jackselbo/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/jackselbo/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [23]:
# Condense event labels
def clean_events(df):
    event_cleaning_dict = {'Sold (MLS)': 'Sold',
                           'Listed': 'Listed',
                           'Listed (Active)': 'Listed',
                           'Sold (MLS) (Sold)': 'Sold',
                           'Sold (Public Records)': 'Sold',
                           'Price Changed': 'Price Changed',
                           'Sold (MLS) (Closed)': 'Sold',
                           'Price Changed (Sold)': 'Sold',
                           'Price Changed (Contract)': 'Price Changed',
                           'Price Changed (Active)': 'Price Changed'
                           }
    df['cleaned_event'] = df['event'].map(event_cleaning_dict)
    df = df.drop('event', axis=1)
    return df

price_data = clean_events(price_data)

In [24]:
# Filter to look at only sale prices
def filter_to_sale_prices(df):
    filtered_df = df[df.cleaned_event == 'Sold'].copy()
    filtered_df = filtered_df.drop('cleaned_event', axis=1)
    return filtered_df

sales_data = filter_to_sale_prices(price_data)

In [25]:
sales_data.shape

(13601, 4)

In [26]:
# Remove duplicates for houses with the same date or price
def remove_duplicates(df):
    df = df[~df.duplicated(subset=['url', 'date'])]
    df = df[~df.duplicated(subset=['url', 'price'])]
    return df

sales_data = remove_duplicates(sales_data)
sales_data.shape

(10472, 4)

In [27]:
sales_data.head()

,date,source,price,url
28,2006-12-01,MIBOR #2637964,624000.0,https://www.redfin.com/IN/Indianapolis/141-S-M...
30,2005-04-01,MIBOR #2420599,425000.0,https://www.redfin.com/IN/Indianapolis/141-S-M...
34,2006-07-01,MIBOR #2555608,517487.0,https://www.redfin.com/IN/Indianapolis/141-S-M...
42,2006-09-01,MIBOR #2620085,590000.0,https://www.redfin.com/IN/Indianapolis/141-S-M...
62,2012-07-01,MIBOR #21144317,339000.0,https://www.redfin.com/IN/Indianapolis/350-N-M...


In [28]:
# leave this out
# Load data and filter to DC metro area
hpi_quarterly_df = pd.read_csv('HPI_master.csv')
hpi_quarterly_df = hpi_quarterly_df[(
    hpi_quarterly_df.place_name == 'Indiana') & (hpi_quarterly_df.hpi_flavor == 'all-transactions')]

In [29]:
# hpi_quarterly_df.head()

In [30]:
# Convert quarters to date time
def quarter_to_month(df):
    quarter_to_month_dict = {
        1: 'January',
        2: 'April',
        3: 'July',
        4: 'October'
    }

    df['period'] = df.period.map(quarter_to_month_dict)
    return df


def quarter_to_datetime(df):
    df['date'] = df.period + "-" + df.yr.astype(str)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    return df


def create_quarterly_df(df):
    df = pd.merge(pd.DataFrame(index=pd.date_range(df.date.min(), datetime.datetime(year=2019, month=1, day=1), freq='MS')),
                  df.set_index('date'), how='outer', left_index=True, right_index=True)
    df = df.ffill()
    return df

hpi_quarterly_df = quarter_to_month(hpi_quarterly_df)
hpi_quarterly_df = quarter_to_datetime(hpi_quarterly_df)
hpi_quarterly_df = hpi_quarterly_df[['place_name', 'index_nsa', 'date']]
hpi_quarterly_by_month = create_quarterly_df(hpi_quarterly_df)

In [31]:
def create_monthly_msa_df(df):
    df = pd.merge(pd.DataFrame(index=pd.date_range(df.date.min(), datetime.datetime(year=2019, month=1, day=1), freq='MS')),
                  df.set_index('date'), how='outer', left_index=True, right_index=True)
    df['place_name'] = df['place_name'].ffill()
    df['index_nsa'] = df['index_nsa'].interpolate()
    return df

hpi_monthly_by_month = create_monthly_msa_df(hpi_quarterly_df)
hpi_monthly_by_month.head()

,place_name,index_nsa
1975-01-01,Indiana,66.160000
1975-02-01,Indiana,64.443333
1975-03-01,Indiana,62.726667
1975-04-01,Indiana,61.010000
1975-05-01,Indiana,61.800000


In [32]:
def combine_sales_msa(sales_df, hpi_df):
    sales_df = sales_df.sort_values(by='date', ascending=True)
    sales_df[['previous_date', 'previous_price']] = sales_df.groupby('url')['date', 'price'].shift(1)
    sales_df = sales_df[(sales_df.date - sales_df.previous_date) > datetime.timedelta(days=180)]
    sales_df = sales_df.dropna()
    sales_df = pd.merge(sales_df, hpi_df[['index_nsa']], how='left', left_on='date', right_index=True)
    sales_df = sales_df.rename({'index_nsa':'hpi_quarterly_new'}, axis=1)
    sales_df = pd.merge(sales_df, hpi_df[['index_nsa']], how='left', left_on='previous_date', right_index=True)
    sales_df = sales_df.rename({'index_nsa':'hpi_quarterly_old'}, axis=1)
    sales_df['expected_price'] = sales_df.previous_price * sales_df.hpi_quarterly_new / sales_df.hpi_quarterly_old
    sales_df['price_diff'] = sales_df.price - sales_df.expected_price
    sales_df['length_of_inv_in_yrs'] = ((sales_df.date - sales_df.previous_date).dt.days/365)
    sales_df['price_diff_per_year'] = sales_df.price_diff / sales_df.length_of_inv_in_yrs
    sales_df['perc_price_diff_per_year'] = sales_df.price_diff_per_year / sales_df.previous_price
    sales_df['perc_growth_per_year'] = sales_df.price_diff / sales_df.previous_price
    sales_df['benchmark_rate_ann'] = (sales_df.expected_price / sales_df.previous_price)**(1/sales_df.length_of_inv_in_yrs)
    sales_df['actual_rate_ann'] = (sales_df.price / sales_df.previous_price) ** (1/sales_df.length_of_inv_in_yrs)
    sales_df['excess_rate_ann'] = sales_df.actual_rate_ann - sales_df.benchmark_rate_ann
    
    return sales_df

sales_quarterly_msa = combine_sales_msa(sales_data, hpi_quarterly_by_month)
sales_monthly_msa = combine_sales_msa(sales_data, hpi_monthly_by_month)

In [33]:
cleaned_house_quarterly_msa_data = pd.merge(sales_quarterly_msa, cleaned_house_data, how='left', left_on='url', right_index=True)
cleaned_house_quarterly_msa_data['age_of_house'] = (cleaned_house_quarterly_msa_data['date'] - cleaned_house_quarterly_msa_data['year built']).dt.days / 365
cleaned_house_quarterly_msa_data['days_since_last_renovation'] = (cleaned_house_quarterly_msa_data['date'] - cleaned_house_quarterly_msa_data['year renovated']).apply(lambda x: np.datetime64('NaT') if x < datetime.timedelta(days=0) else x).dt.days

In [34]:
cleaned_house_monthly_msa_data = pd.merge(sales_monthly_msa, cleaned_house_data, how='left', left_on='url', right_index=True)
cleaned_house_monthly_msa_data['age_of_house'] = (cleaned_house_monthly_msa_data['date'] - cleaned_house_monthly_msa_data['year built']).dt.days / 365
cleaned_house_monthly_msa_data['days_since_last_renovation'] = (cleaned_house_monthly_msa_data['date'] - cleaned_house_monthly_msa_data['year renovated']).apply(lambda x: np.datetime64('NaT') if x < datetime.timedelta(days=0) else x).dt.days

In [35]:
cleaned_house_quarterly_msa_data.head(10)['excess_rate_ann']

2138     5.108439
44684   -0.017151
20298    5.073598
20298    3.594064
20298    5.681423
20298    5.074432
51731   -0.228909
51731    0.255918
49349    0.102788
49349   -0.480071
Name: excess_rate_ann, dtype: float64

In [36]:
zip_hpi_df = pd.read_csv('HPI_AT_ZIP5.csv', dtype='str')
zip_hpi_df = zip_hpi_df.drop(['HPI with 1990 base', 'HPI with 2000 base', 'Unnamed: 6', 'Unnamed: 7', 'Annual Change (%)'], axis=1)
zip_hpi_df['HPI'] = pd.to_numeric(zip_hpi_df.HPI, errors='coerce')
zip_hpi_df = zip_hpi_df.dropna()
zip_hpi_df['date'] = pd.to_datetime(zip_hpi_df.Year, infer_datetime_format=True)
zip_hpi_df = zip_hpi_df.drop('Year', axis=1)
zip_hpi_df = zip_hpi_df.rename({'Five-Digit ZIP Code':'zip_code'}, axis=1)
zip_hpi_df = zip_hpi_df.rename({'HPI':'index_nsa'}, axis=1)
zip_hpi_df.head()

,zip_code,index_nsa,date
0,01001,100.00,1985-01-01
1,01001,114.44,1986-01-01
2,01001,138.69,1987-01-01
3,01001,163.18,1988-01-01
4,01001,164.44,1989-01-01


In [37]:
zip_hpi_annual = zip_hpi_df.groupby('zip_code').apply(create_quarterly_df).drop('zip_code', axis=1).reset_index()
zip_hpi_annual = zip_hpi_annual.rename({'level_1':'date'}, axis=1)
zip_hpi_annual = zip_hpi_annual.rename({'HPI':'index_nsa'}, axis=1)
zip_hpi_annual.head()

,zip_code,date,index_nsa
0,01001,1985-01-01,100.0
1,01001,1985-02-01,100.0
2,01001,1985-03-01,100.0
3,01001,1985-04-01,100.0
4,01001,1985-05-01,100.0


In [38]:
def create_monthly_zip_df(df):
    df = pd.merge(pd.DataFrame(index=pd.date_range(df.date.min(), datetime.datetime(year=2019, month=1, day=1), freq='MS')),
                  df.set_index('date'), how='outer', left_index=True, right_index=True)
    df['index_nsa'] = df['index_nsa'].interpolate()
#     df = df.rename({'HPI':'index_nsa'}, axis=1)
    return df


zip_hpi_monthly = zip_hpi_df.groupby('zip_code')\
    .apply(create_monthly_zip_df)\
    .drop('zip_code', axis=1)\
    .reset_index()\
    .rename({'level_1': 'date'}, axis=1)

zip_hpi_monthly.head()

,zip_code,date,index_nsa
0,01001,1985-01-01,100.000000
1,01001,1985-02-01,101.203333
2,01001,1985-03-01,102.406667
3,01001,1985-04-01,103.610000
4,01001,1985-05-01,104.813333


In [39]:
def combine_sales_zip(sales_df, hpi_df):
    hpi_df['zip_code'] = hpi_df['zip_code'].astype(int)
    sales_df = sales_df.sort_values(by='date', ascending=True)
    sales_df[['previous_date', 'previous_price']] = sales_df.groupby('url')['date', 'price'].shift(1)
    sales_df = sales_df[(sales_df.date - sales_df.previous_date) > datetime.timedelta(days=180)]
    sales_df = sales_df.dropna(subset=['previous_price'])
    sales_df = pd.merge(sales_df, hpi_df[['index_nsa', 'date', 'zip_code']], how='left', left_on=['postalcode', 'date'], right_on=['zip_code', 'date'])
    sales_df = sales_df.rename({'index_nsa':'hpi_annual_new'}, axis=1)
    sales_df = pd.merge(sales_df, hpi_df[['index_nsa', 'date', 'zip_code']], how='left', left_on=['postalcode', 'previous_date'], right_on=['zip_code', 'date'])
    sales_df = sales_df.drop(['date_y', 'zip_code_y'], axis=1)
    sales_df = sales_df.rename({'index_nsa':'hpi_annual_old', 'date_x':'date'}, axis=1)
    sales_df['expected_price'] = sales_df.previous_price * sales_df.hpi_annual_new / sales_df.hpi_annual_old
    sales_df['price_diff'] = sales_df.price - sales_df.expected_price
    sales_df['length_of_inv_in_yrs'] = ((sales_df.date - sales_df.previous_date).dt.days/365)
    sales_df['price_diff_per_year'] = sales_df.price_diff / sales_df.length_of_inv_in_yrs
    sales_df['perc_price_diff_per_year'] = sales_df.price_diff_per_year / sales_df.previous_price
    sales_df['perc_growth_per_year'] = sales_df.price_diff / sales_df.previous_price
    sales_df['benchmark_rate_ann'] = (sales_df.expected_price / sales_df.previous_price)**(1/sales_df.length_of_inv_in_yrs)
    sales_df['actual_rate_ann'] = (sales_df.price / sales_df.previous_price) ** (1/sales_df.length_of_inv_in_yrs)
    sales_df['excess_rate_ann'] = sales_df.actual_rate_ann - sales_df.benchmark_rate_ann
    
    return sales_df

In [40]:
cleaned_house_annually_zip_data = pd.merge(sales_data, cleaned_house_data, how='left', left_on='url', right_index=True)
cleaned_house_annually_zip_data['age_of_house'] = (cleaned_house_annually_zip_data['date'] - cleaned_house_annually_zip_data['year built']).dt.days / 365
cleaned_house_annually_zip_data['days_since_last_renovation'] = (cleaned_house_annually_zip_data['date'] - cleaned_house_annually_zip_data['year renovated']).apply(lambda x: np.datetime64('NaT') if x < datetime.timedelta(days=0) else x).dt.days
cleaned_house_annually_zip_data = combine_sales_zip(cleaned_house_annually_zip_data, zip_hpi_annual)
cleaned_house_annually_zip_data = cleaned_house_annually_zip_data.dropna(subset=['excess_rate_ann'])

In [41]:
cleaned_house_monthly_zip_data = pd.merge(sales_data, cleaned_house_data, how='left', left_on='url', right_index=True)
cleaned_house_monthly_zip_data['age_of_house'] = (cleaned_house_monthly_zip_data['date'] - cleaned_house_monthly_zip_data['year built']).dt.days / 365
cleaned_house_monthly_zip_data['days_since_last_renovation'] = (cleaned_house_monthly_zip_data['date'] - cleaned_house_monthly_zip_data['year renovated']).apply(lambda x: np.datetime64('NaT') if x < datetime.timedelta(days=0) else x).dt.days
cleaned_house_monthly_zip_data = combine_sales_zip(cleaned_house_monthly_zip_data, zip_hpi_monthly)
cleaned_house_monthly_zip_data = cleaned_house_monthly_zip_data.dropna(subset=['excess_rate_ann'])
cleaned_house_monthly_zip_data = cleaned_house_monthly_zip_data.rename(columns={'zip_code_x': 'zip_code'})

In [42]:
cleaned_house_monthly_zip_data.columns

Index(['date', 'source', 'price', 'url', 'streetaddress', 'addresslocality',
       'addressregion', 'postalcode', 'community', 'latitude', 'longitude',
       'beds', 'baths', 'stories', 'lot size', 'year built', 'year renovated',
       'transit score®', 'bike score®', 'walk score®', 'hoa dues',
       'finished sq. ft.', 'total sq. ft.', 'style_Condo/Co-op',
       'style_MasterPlanned', 'style_Multi-Family (2-4 Unit)', 'style_Other',
       'style_Single Family Residential', 'style_Vacant Land', 'description',
       'age_of_house', 'days_since_last_renovation', 'previous_date',
       'previous_price', 'hpi_annual_new', 'zip_code', 'hpi_annual_old',
       'expected_price', 'price_diff', 'length_of_inv_in_yrs',
       'price_diff_per_year', 'perc_price_diff_per_year',
       'perc_growth_per_year', 'benchmark_rate_ann', 'actual_rate_ann',
       'excess_rate_ann'],
      dtype='object')

In [43]:
cleaned_house_monthly_zip_data.head()


,date,source,price,url,streetaddress,addresslocality,addressregion,postalcode,community,latitude,...,hpi_annual_old,expected_price,price_diff,length_of_inv_in_yrs,price_diff_per_year,perc_price_diff_per_year,perc_growth_per_year,benchmark_rate_ann,actual_rate_ann,excess_rate_ann
0,2001-03-01,Public Records,117040.0,https://www.redfin.com/IN/Indianapolis/1528-E-...,1528 E Ohio St,"Indianapolis,",IN,46201,Center NE,39.769241,...,250.320833,30639.079848,86400.920152,0.747945,115517.713757,3.850590,2.880031,1.028583,6.172338,5.143755
1,2001-06-01,MIBOR #2112902,150000.0,https://www.redfin.com/IN/Indianapolis/5709-E-...,5709 Sherman Ave,"Indianapolis,",IN,46220,Washington,39.858151,...,458.810000,150377.109043,-377.109043,0.580822,-649.267928,-0.004422,-0.002568,1.041930,1.037435,-0.004495
2,2001-07-01,Public Records,136990.0,https://www.redfin.com/IN/Indianapolis/5114-Ca...,5114 Carvel Ave,"Indianapolis,",IN,46205,Washington,39.845956,...,353.580000,57722.631936,79267.368064,0.495890,159848.559908,2.861593,1.419036,1.068382,6.104261,5.035880
3,2001-07-01,Public Records,79451.0,https://www.redfin.com/IN/Indianapolis/6024-Ru...,6024 Ruskin Pl,"Indianapolis,",IN,46224,Wayne,39.823184,...,211.053333,59926.890517,19524.109483,1.167123,16728.403665,0.288421,0.336623,1.028398,1.309487,0.281088
4,2001-07-01,Public Records,202891.0,https://www.redfin.com/IN/Indianapolis/6941-Bl...,6941 Bluffgrove Ln,"Indianapolis,",IN,46278,Pike,39.880035,...,170.745000,183492.624908,19398.375092,1.000000,19398.375092,0.109829,0.109829,1.038888,1.148717,0.109829


In [44]:
export1 = cleaned_house_monthly_zip_data.to_csv('~/Desktop/cleaned.csv',index=False)

In [45]:
cleaned = cleaned_house_monthly_zip_data
cleaned['etaZSCORE'] = cleaned[['excess_rate_ann']].apply(zscore).abs()
extreme = cleaned[cleaned['etaZSCORE']<.45]
extreme.shape

(3409, 47)

In [46]:
cleaned.shape

(3579, 47)

In [47]:
export2 = extreme.to_csv('~/Desktop/pruned.csv',index=False)

In [48]:
excessdf = pd.DataFrame(cleaned.groupby('zip_code').excess_rate_ann.mean())
pricedf = pd.DataFrame(cleaned.groupby('zip_code').price.mean())
latdf = pd.DataFrame(cleaned.groupby('zip_code').latitude.mean())
longdf = pd.DataFrame(cleaned.groupby('zip_code').longitude.mean())
zipdf = excessdf.join(pricedf, how='outer').join(latdf, how='outer').join(longdf, how='outer')
zipdf['zip'] = zipdf.index.values
zipdf['zip'] = [int(i) for i in zipdf['zip']]
zipdf

,excess_rate_ann,price,latitude,longitude,zip
zip_code,,,,,
46123.0,-0.010493,132000.000000,39.783764,-86.208968,46123
46201.0,0.922340,75409.862944,39.777020,-86.108951,46201
46202.0,0.545762,283994.330579,39.785374,-86.149064,46202
46203.0,0.743958,117915.925781,39.744213,-86.121924,46203
46205.0,0.367303,163976.553957,39.825451,-86.142835,46205
46208.0,0.445624,154076.860000,39.827332,-86.166397,46208
46214.0,0.046357,122807.328767,39.799701,-86.292804,46214
46217.0,0.066773,131448.113402,39.664006,-86.187046,46217
46218.0,0.308108,31694.899083,39.808576,-86.106145,46218


In [49]:
export4 = zipdf.to_csv('~/Desktop/attempt2.csv',index=True)

In [50]:
cleaned['finished sq. ft.']

0       2209.0
1       2240.0
2       1016.0
3          NaN
4          NaN
5          NaN
6          NaN
7        960.0
8       2712.0
9          NaN
10         NaN
11      1680.0
12         NaN
13      2242.0
14         NaN
15         NaN
16         NaN
17         NaN
18         NaN
19         NaN
20         NaN
21         NaN
22         NaN
23         NaN
24         NaN
25      4567.0
26         NaN
27      3983.0
28      2028.0
29         NaN
         ...  
3596       NaN
3597       NaN
3598       NaN
3599       NaN
3600       NaN
3601       NaN
3602       NaN
3603       NaN
3604       NaN
3605       NaN
3606       NaN
3607       NaN
3608       NaN
3609       NaN
3610       NaN
3611       NaN
3612       NaN
3613       NaN
3614       NaN
3615       NaN
3616       NaN
3617       NaN
3618       NaN
3619       NaN
3620       NaN
3621       NaN
3622       NaN
3623       NaN
3624       NaN
3625       NaN
Name: finished sq. ft., Length: 3579, dtype: float64

In [51]:
print("hi")

hi


In [52]:
regression1 = cleaned_house_monthly_zip_data.select_dtypes(include=['float64','int64'])
regression1.head()

,price,postalcode,latitude,longitude,beds,baths,stories,lot size,transit score®,bike score®,...,expected_price,price_diff,length_of_inv_in_yrs,price_diff_per_year,perc_price_diff_per_year,perc_growth_per_year,benchmark_rate_ann,actual_rate_ann,excess_rate_ann,etaZSCORE
0,117040.0,46201,39.769241,-86.132359,3.0,NaN,1.0,4008.0,38.0,74.0,...,30639.079848,86400.920152,0.747945,115517.713757,3.850590,2.880031,1.028583,6.172338,5.143755,2.533360
1,150000.0,46220,39.858151,-86.104695,3.0,NaN,1.0,NaN,25.0,39.0,...,150377.109043,-377.109043,0.580822,-649.267928,-0.004422,-0.002568,1.041930,1.037435,-0.004495,0.144424
2,136990.0,46205,39.845956,-86.139699,2.0,NaN,1.0,NaN,32.0,90.0,...,57722.631936,79267.368064,0.495890,159848.559908,2.861593,1.419036,1.068382,6.104261,5.035880,2.477250
3,79451.0,46224,39.823184,-86.266031,3.0,NaN,1.0,6882.0,37.0,54.0,...,59926.890517,19524.109483,1.167123,16728.403665,0.288421,0.336623,1.028398,1.309487,0.281088,0.004117
4,202891.0,46278,39.880035,-86.280468,3.0,1.5,2.0,NaN,24.0,63.0,...,183492.624908,19398.375092,1.000000,19398.375092,0.109829,0.109829,1.038888,1.148717,0.109829,0.084961


In [53]:
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

In [66]:

X_train, X_test, y_train, y_test = train_test_split(regression1.drop('excess_rate_ann', axis = 1), regression1['excess_rate_ann'],test_size = .25, random_state=29)
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.08, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.75)

In [67]:
predictions = xgb.predict(X_test)
print(explained_variance_score(predictions,y_test))

0.9931318010756041


In [58]:
predictions

array([ 1.16810799e-02, -6.65987134e-02,  1.08297777e+00,  8.94457102e-03,
        4.24273014e-02,  1.78682804e-01,  4.05312777e-02, -5.68028688e-02,
        2.31625438e-02,  3.00139487e-02,  8.79226923e-02,  6.66523874e-02,
       -1.83540583e-02,  1.35990381e-01,  4.57378030e-02,  1.58950686e-02,
        1.35813355e-02,  7.29565918e-02,  2.06539035e-02,  2.45499611e-02,
        1.67926252e-02,  3.94740194e-01, -2.00661778e-01,  1.96791470e-01,
       -6.25849962e-02,  6.22370243e-02, -9.82323885e-02,  4.68224287e-02,
        6.97392225e-03, -2.63898253e-01, -1.41963303e-01,  1.82654530e-01,
       -1.73313022e-02,  9.57719207e-01, -5.72189689e-02,  2.60905325e-02,
        7.18736649e-03,  1.37463599e-01, -8.24340582e-02, -1.82659030e-02,
       -7.85368681e-03, -3.18974257e-03,  4.31472063e-03,  9.31158662e-02,
        8.10804069e-02, -7.85614848e-02,  3.89862657e-02,  6.18859529e-02,
        4.56993580e-02,  4.64222431e-02, -3.34106624e-01,  2.85652280e-03,
        1.11661851e-02, -

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(regression1.drop('excess_rate_ann', axis = 1), regression1['excess_rate_ann'],test_size = .25)


In [ ]:
m = LinearRegression()

In [ ]:
m.fit(X_train,y_train)
print(m.score(X_train,y_train))
m.coef_